<a href="https://colab.research.google.com/github/vikramkrishnan9885/MyColab/blob/master/NMT_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip show tensorflow

Name: tensorflow
Version: 1.15.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: six, termcolor, protobuf, google-pasta, keras-preprocessing, tensorboard, wrapt, numpy, wheel, absl-py, keras-applications, astor, opt-einsum, tensorflow-estimator, grpcio, gast
Required-by: fancyimpute


In [67]:
!pip uninstall tensorflow

Uninstalling tensorflow-2.3.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.3.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
y
y
y
y
  Successfully uninstalled tensorflow-2.3.0
y


In [68]:
!pip install tensorflow==1.15

     |████████████████████████████████| 412.3MB 35kB/s 
y
     |████████████████████████████████| 51kB 5.2MB/s 
     |████████████████████████████████| 3.8MB 42.2MB/s 
     |████████████████████████████████| 512kB 41.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=bb25ced7a722a3c4eea0505b4c29c1c6a554852bde03ff0c9f4492ff48c8d3d6
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled te

In [1]:
%matplotlib inline
from __future__ import print_function
import collections
import math
import numpy as np
import os
import random
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import zipfile
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
#import tensorflow as tf
from PIL import Image
from collections import Counter
import csv
import matplotlib.gridspec as gridspec


from nltk.translate.bleu_score import corpus_bleu
import nltk

import urllib.request

Instructions for updating:
non-resource variables are not supported in the long term


# Helper functions for word2vec

In [2]:
sentence_cursors = None
tot_sentences = None
src_max_sent_length, tgt_max_sent_length = 0, 0
src_dictionary, tgt_dictionary = {}, {}
src_reverse_dictionary, tgt_reverse_dictionary = {},{}
train_inputs, train_outputs = None, None
embedding_size = None # Dimension of the embedding vector.
vocabulary_size = None

def define_data_and_hyperparameters(
        _tot_sentences, _src_max, _tgt_max, _src_dict, _tgt_dict,
        _src_rev_dict, _tgt_rev_dict, _tr_inp, _tr_out, _emb_size, _vocab_size):
    global tot_sentences, sentence_cursors
    global src_max_sent_length, tgt_max_sent_length
    global src_dictionary, tgt_dictionary
    global src_reverse_dictionary, tgt_reverse_dictionary
    global train_inputs, train_outputs
    global embedding_size, vocabulary_size

    embedding_size = _emb_size
    vocabulary_size = _vocab_size
    src_max_sent_length, tgt_max_sent_length = _src_max, _tgt_max

    src_dictionary = _src_dict
    tgt_dictionary = _tgt_dict

    src_reverse_dictionary = _src_rev_dict
    tgt_reverse_dictionary = _tgt_rev_dict

    train_inputs = _tr_inp
    train_outputs = _tr_out

    tot_sentences = _tot_sentences
    sentence_cursors = [0 for _ in range(tot_sentences)]


def generate_batch_for_word2vec(batch_size, window_size, is_source):
    # window_size is the amount of words we're looking at from each side of a given word
    # creates a single batch
    global sentence_cursors
    global src_dictionary, tgt_dictionary
    global train_inputs, train_outputs
    span = 2 * window_size + 1  # [ skip_window target skip_window ]

    batch = np.ndarray(shape=(batch_size, span - 1), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    # e.g if skip_window = 2 then span = 5
    # span is the length of the whole frame we are considering for a single word (left + word + right)
    # skip_window is the length of one side

    sentence_ids_for_batch = np.random.randint(0, tot_sentences, batch_size)

    for b_i in range(batch_size):
        sent_id = sentence_ids_for_batch[b_i]

        if is_source:
            buffer = train_inputs[sent_id, sentence_cursors[sent_id]:sentence_cursors[sent_id] + span]
        else:
            buffer = train_outputs[sent_id, sentence_cursors[sent_id]:sentence_cursors[sent_id] + span]
        assert buffer.size == span, 'Buffer length (%d), Current data index (%d), Span(%d)' % (
        buffer.size, sentence_cursors[sent_id], span)
        # If we only have EOS tokesn in the sampled text, we sample a new one
        if is_source:
            while np.all(buffer == src_dictionary['</s>']):
                # reset the sentence_cursors for that cap_id
                sentence_cursors[sent_id] = 0
                # sample a new cap_id
                sent_id = np.random.randint(0, tot_sentences)
                buffer = train_inputs[sent_id, sentence_cursors[sent_id]:sentence_cursors[sent_id] + span]
        else:
            while np.all(buffer == tgt_dictionary['</s>']):
                # reset the sentence_cursors for that cap_id
                sentence_cursors[sent_id] = 0
                # sample a new cap_id
                sent_id = np.random.randint(0, tot_sentences)
                buffer = train_outputs[sent_id, sentence_cursors[sent_id]:sentence_cursors[sent_id] + span]

        # fill left and right sides of batch
        batch[b_i, :window_size] = buffer[:window_size]
        batch[b_i, window_size:] = buffer[window_size + 1:]

        labels[b_i, 0] = buffer[window_size]

        # increase the corresponding index
        if is_source:
            sentence_cursors[sent_id] = (sentence_cursors[sent_id] + 1) % (src_max_sent_length - span)
        else:
            sentence_cursors[sent_id] = (sentence_cursors[sent_id] + 1) % (tgt_max_sent_length - span)

    assert batch.shape[0] == batch_size and batch.shape[1] == span - 1
    return batch, labels


def print_some_batches():
    global sentence_cursors, tot_sentences
    global src_reverse_dictionary

    for window_size in [1, 2]:
        sentence_cursors = [0 for _ in range(tot_sentences)]
        batch, labels = generate_batch_for_word2vec(batch_size=8, window_size=window_size, is_source=True)
        print('\nwith window_size = %d:' % (window_size))
        print('    batch:', [[src_reverse_dictionary[bii] for bii in bi] for bi in batch])
        print('    labels:', [src_reverse_dictionary[li] for li in labels.reshape(8)])

    sentence_cursors = [0 for _ in range(tot_sentences)]

batch_size, window_size = None, None
valid_size, valid_window, valid_examples = None, None, None
num_sampled = None

train_dataset, train_labels = None, None
valid_dataset = None

softmax_weights, softmax_biases = None, None

loss, optimizer, similarity, normalized_embeddings = None, None, None, None

def define_word2vec_tensorflow(batch_size):

    global embedding_size, window_size
    global	valid_size, valid_window, valid_examples
    global num_sampled
    global train_dataset, train_labels
    global valid_dataset
    global softmax_weights, softmax_biases
    global loss, optimizer, similarity
    global vocabulary_size, embedding_size
    global normalized_embeddings


    window_size = 2  # How many words to consider left and right.
    # We pick a random validation set to sample nearest neighbors. here we limit the
    # validation samples to the words that have a low numeric ID, which by
    # construction are also the most frequent.
    valid_size = 20  # Random set of words to evaluate similarity on.
    valid_window = 100  # Only pick dev samples in the head of the distribution.
    # pick 16 samples from 100
    valid_examples = np.array(np.random.randint(0, valid_window, valid_size // 2))
    valid_examples = np.append(valid_examples, np.random.randint(1000, 1000 + valid_window, valid_size // 2))
    num_sampled = 32  # Number of negative examples to sample.

    tf.compat.v1.reset_default_graph()

    # Input data.
    train_dataset = tf.compat.v1.placeholder(tf.compat.v1.int32, shape=[batch_size, 2 * window_size])
    train_labels = tf.compat.v1.placeholder(tf.compat.v1.int32, shape=[batch_size, 1])
    valid_dataset = tf.compat.v1.constant(valid_examples, dtype=tf.compat.v1.int32)

    # Variables.
    # embedding, vector for each word in the vocabulary
    embeddings = tf.compat.v1.Variable(tf.compat.v1.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0, dtype=tf.compat.v1.float32))
    softmax_weights = tf.compat.v1.Variable(tf.compat.v1.truncated_normal([vocabulary_size, embedding_size],
                                                      stddev=1.0 / math.sqrt(embedding_size), dtype=tf.compat.v1.float32))
    softmax_biases = tf.compat.v1.Variable(tf.compat.v1.zeros([vocabulary_size], dtype=tf.compat.v1.float32))

    # Model.
    # Look up embeddings for inputs.
    # this might efficiently find the embeddings for given ids (traind dataset)
    # manually doing this might not be efficient given there are 50000 entries in embeddings
    stacked_embedings = None
    print('Defining %d embedding lookups representing each word in the context' % (2 * window_size))
    for i in range(2 * window_size):
        embedding_i = tf.compat.v1.nn.embedding_lookup(embeddings, train_dataset[:, i])
        x_size, y_size = embedding_i.get_shape().as_list()
        if stacked_embedings is None:
            stacked_embedings = tf.compat.v1.reshape(embedding_i, [x_size, y_size, 1])
        else:
            stacked_embedings = tf.compat.v1.concat(axis=2,
                                          values=[stacked_embedings, tf.compat.v1.reshape(embedding_i, [x_size, y_size, 1])])

    assert stacked_embedings.get_shape().as_list()[2] == 2 * window_size
    print("Stacked embedding size: %s" % stacked_embedings.get_shape().as_list())
    mean_embeddings = tf.compat.v1.reduce_mean(stacked_embedings, 2, keepdims=False)
    print("Reduced mean embedding size: %s" % mean_embeddings.get_shape().as_list())

    # Compute the softmax loss, using a sample of the negative labels each time.
    # inputs are embeddings of the train words
    # with this loss we optimize weights, biases, embeddings

    loss = tf.compat.v1.reduce_mean(
        tf.compat.v1.nn.sampled_softmax_loss(weights=softmax_weights, biases=softmax_biases, inputs=mean_embeddings,
                                   labels=train_labels, num_sampled=num_sampled, num_classes=vocabulary_size))

    # Optimizer.
    # Note: The optimizer will optimize the softmax_weights AND the embeddings.
    optimizer = tf.compat.v1.train.AdamOptimizer(0.001).minimize(loss)

    # Compute the similarity between minibatch examples and all embeddings.
    # We use the cosine distance:
    norm = tf.compat.v1.sqrt(tf.compat.v1.reduce_sum(tf.compat.v1.square(embeddings), 1, keepdims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.compat.v1.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.compat.v1.matmul(valid_embeddings, tf.compat.v1.transpose(normalized_embeddings))


def run_word2vec_source(batch_size):
    global embedding_size, window_size
    global valid_size, valid_window, valid_examples
    global num_sampled
    global train_dataset, train_labels
    global valid_dataset
    global softmax_weights, softmax_biases
    global loss, optimizer, similarity, normalized_embeddings
    global src_reverse_dictionary
    global vocabulary_size, embedding_size

    num_steps = 100001

    config=tf.compat.v1.ConfigProto(allow_soft_placement=True) 
    config.gpu_options.allow_growth = True	
    	
    with tf.compat.v1.Session(config=config) as session:
        tf.compat.v1.global_variables_initializer().run()
        print('Initialized')
        average_loss = 0
        for step in range(num_steps):

            batch_data, batch_labels = generate_batch_for_word2vec(batch_size, window_size, is_source=True)
            feed_dict = {train_dataset: batch_data, train_labels: batch_labels}
            _, l = session.run([optimizer, loss], feed_dict=feed_dict)
            average_loss += l
            if (step + 1) % 2000 == 0:
                if step > 0:
                    average_loss = average_loss / 2000
                    # The average loss is an estimate of the loss over the last 2000 batches.
                print('Average loss at step %d: %f' % (step + 1, average_loss))
                average_loss = 0
            # note that this is expensive (~20% slowdown if computed every 500 steps)
            if (step + 1) % 10000 == 0:
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = src_reverse_dictionary[valid_examples[i]]
                    top_k = 8  # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                    log = 'Nearest to %s:' % valid_word
                    for k in range(top_k):
                        close_word = src_reverse_dictionary[nearest[k]]
                        log = '%s %s,' % (log, close_word)
                    print(log)
        cbow_final_embeddings = normalized_embeddings.eval()

    np.save('de-embeddings.npy', cbow_final_embeddings)

def run_word2vec_target(batch_size):
    global embedding_size, window_size
    global valid_size, valid_window, valid_examples
    global num_sampled
    global train_dataset, train_labels
    global valid_dataset
    global softmax_weights, softmax_biases
    global loss, optimizer, similarity, normalized_embeddings
    global tgt_reverse_dictionary
    global vocabulary_size, embedding_size

    num_steps = 100001
    
    config=tf.compat.v1.ConfigProto(allow_soft_placement=True) 
    config.gpu_options.allow_growth = True	
    with tf.compat.v1.Session(config=config) as session:
        tf.compat.v1.global_variables_initializer().run()
        print('Initialized')
        average_loss = 0
        for step in range(num_steps):

            batch_data, batch_labels = generate_batch_for_word2vec(batch_size, window_size, is_source=False)
            feed_dict = {train_dataset: batch_data, train_labels: batch_labels}
            _, l = session.run([optimizer, loss], feed_dict=feed_dict)
            average_loss += l
            if (step + 1) % 2000 == 0:
                if step > 0:
                    average_loss = average_loss / 2000
                    # The average loss is an estimate of the loss over the last 2000 batches.
                print('Average loss at step %d: %f' % (step + 1, average_loss))
                average_loss = 0
            # note that this is expensive (~20% slowdown if computed every 500 steps)
            if (step + 1) % 10000 == 0:
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = tgt_reverse_dictionary[valid_examples[i]]
                    top_k = 8  # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                    log = 'Nearest to %s:' % valid_word
                    for k in range(top_k):
                        close_word = tgt_reverse_dictionary[nearest[k]]
                        log = '%s %s,' % (log, close_word)
                    print(log)
        cbow_final_embeddings = normalized_embeddings.eval()

    np.save('en-embeddings.npy', cbow_final_embeddings)

In [3]:
print('Beginning file download with urllib2...')

url_train_de = 'https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/train.de'
urllib.request.urlretrieve(url_train_de, 'train.de')

url_train_en = 'https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/train.en'
urllib.request.urlretrieve(url_train_en, 'train.en')

url_vocab_de = 'https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/vocab.50K.de'
urllib.request.urlretrieve(url_vocab_de, 'vocab.50K.de')

url_vocab_en = 'https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/vocab.50K.en'
urllib.request.urlretrieve(url_vocab_en, 'vocab.50K.en')

Beginning file download with urllib2...


('vocab.50K.en', <http.client.HTTPMessage at 0x7f41644f9ba8>)

# Data prep

## Load vocabs

In [4]:
#  SEE THE VOCAB CLASSES IN THE PYTORCH EXAMPLES
# Building source language vocabulary

# Contains word string -> ID mapping
src_dictionary = dict()

# Read the vocabulary file
with open('vocab.50K.de', encoding='utf-8') as f:
    # Read and store every line
    for line in f:
        #we are discarding last char as it is new line char
        src_dictionary[line[:-1]] = len(src_dictionary)

# Build a reverse dictionary with the mapping ID -> word string
src_reverse_dictionary = dict(zip(src_dictionary.values(),src_dictionary.keys()))

# Print some of the words in the dictionary
print('Source')
print('\t',list(src_dictionary.items())[:10])
print('\t',list(src_reverse_dictionary.items())[:10])
print('\t','Vocabulary size: ', len(src_dictionary))

Source
	 [('<unk>', 0), ('<s>', 1), ('</s>', 2), (',', 3), ('.', 4), ('die', 5), ('der', 6), ('und', 7), ('in', 8), ('zu', 9)]
	 [(0, '<unk>'), (1, '<s>'), (2, '</s>'), (3, ','), (4, '.'), (5, 'die'), (6, 'der'), (7, 'und'), (8, 'in'), (9, 'zu')]
	 Vocabulary size:  50000


In [5]:
# Building target language vocabulary

# Contains word string -> ID mapping
tgt_dictionary = dict()

# Read the vocabulary file
with open('vocab.50K.en', encoding='utf-8') as f:
    # Read and store every line
    for line in f:
        #we are discarding last char as it is new line char
        tgt_dictionary[line[:-1]] = len(tgt_dictionary)

# Build a reverse dictionary with the mapping ID -> word string
tgt_reverse_dictionary = dict(zip(tgt_dictionary.values(),tgt_dictionary.keys()))

# Print some of the words in the dictionary
print('Target')
print('\t',list(tgt_dictionary.items())[:10])
print('\t',list(tgt_reverse_dictionary.items())[:10])
print('\t','Vocabulary size: ', len(tgt_dictionary))

# Each language has 50000 words
vocabulary_size = 50000

Target
	 [('<unk>', 0), ('<s>', 1), ('</s>', 2), ('the', 3), (',', 4), ('.', 5), ('of', 6), ('and', 7), ('to', 8), ('in', 9)]
	 [(0, '<unk>'), (1, '<s>'), (2, '</s>'), (3, 'the'), (4, ','), (5, '.'), (6, 'of'), (7, 'and'), (8, 'to'), (9, 'in')]
	 Vocabulary size:  50000


## Load training and testing data

In [6]:
# Contains the training sentences
source_sent = [] # Input
target_sent = [] # Output

# Contains the testing sentences
test_source_sent = [] # Input
test_target_sent = [] # Output

# We grab around 100 lines of data that are interleaved 
# in the first 50000 sentences
test_indices = [l_i for l_i in range(50,50001,500)]

# Read the source data file and read the first 250,000 lines (except first 50)
with open('train.de', encoding='utf-8') as f:
    for l_i, line in enumerate(f):
        # discarding first 50 translations as there was some
        # english to english mappings found in the first few lines. which are wrong
        if l_i<50:
            continue
        
        if len(source_sent)<250000 and l_i not in test_indices:
            source_sent.append(line)
        elif l_i in test_indices:
            test_source_sent.append(line)
        
# Read the target data file and read the first 250,000 lines (except first 50)            
with open('train.en', encoding='utf-8') as f:
    for l_i, line in enumerate(f):
        # discarding first 50 translations as there was some
        # english to english mappings found in the first few lines. which are wrong
        if l_i<50:
            continue
        
        if len(target_sent)<250000 and l_i not in test_indices:
            target_sent.append(line)
        elif l_i in test_indices:
            test_target_sent.append(line)
        
# Make sure we extracted same number of both extracted source and target sentences         
assert len(source_sent)==len(target_sent),'Source: %d, Target: %d'%(len(source_sent),len(target_sent))

# Print some source sentences
print('Sample translations (%d)'%len(source_sent))
for i in range(0,250000,10000):
    print('(',i,') DE: ', source_sent[i])
    print('(',i,') EN: ', target_sent[i])

# Print some target sentences
print('Sample test translations (%d)'%len(test_source_sent))
for i in range(0,100,10):
    print('DE: ', test_source_sent[i])
    print('EN: ', test_target_sent[i])

Sample translations (250000)
( 0 ) DE:  Hier erfahren Sie , wie Sie Creative Suite 2 und Creative Suite 3 am besten zusammen mit QuarkXPress nutzen können .

( 0 ) EN:  Here , you ’ ll find out how Creative Suite users can get the best possible interaction with QuarkXPress .

( 10000 ) DE:  Für die sehr günstigen Wochen- und Monatskarten ( 1 Monat ca.

( 10000 ) EN:  It is THE trendy area of Marseille .

( 20000 ) DE:  Freuen Sie sich auf die romantische Atmosphäre in den Zimmern und Apartments .

( 20000 ) EN:  Enjoy the romantic atmosphere of one of the guest rooms or apartments .

( 30000 ) DE:  Zu zwiespältig sind Dr. Gutherzens Erfahrungen aus frühen Studententagen verlaufen , in denen er sich in die Gefielde von durch Heidegger geprägten Autor / innen begeben hat und dort ständig mit strengem Blick darauf verwiesen wurde , er habe bestimmte Theorieressourcen und Gedankengebäude einfach noch nicht gründlich genug verstanden und könne deshalb nicht begreifen , warum seine Einwände 

## Preprocess the data

### Split data into token

In [7]:
# Keep track of how many unknown words were encountered
src_unk_count, tgt_unk_count = 0, 0

def split_to_tokens(sent,is_source):
    '''
    This function takes in a sentence (source or target)
    and preprocess the sentency with various steps (e.g. removing punctuation)
    '''
    
    global src_unk_count, tgt_unk_count

    # Remove punctuation and new-line chars
    sent = sent.replace(',',' ,')
    sent = sent.replace('.',' .')
    sent = sent.replace('\n',' ') 
    
    sent_toks = sent.split(' ')
    for t_i, tok in enumerate(sent_toks):
        if is_source:
            # src_dictionary contain the word -> word ID mapping for source vocabulary
            if tok not in src_dictionary.keys():
                if not len(tok.strip())==0:
                    sent_toks[t_i] = '<unk>'
                    src_unk_count += 1
        else:
            # tgt_dictionary contain the word -> word ID mapping for target vocabulary
            if tok not in tgt_dictionary.keys():
                if not len(tok.strip())==0:
                    sent_toks[t_i] = '<unk>'
                    #print(tok)
                    tgt_unk_count += 1
    return sent_toks

In [8]:
# Let us first look at some statistics of the sentences
# Train - source data
source_len = []
source_mean, source_std = 0,0
for sent in source_sent:
    source_len.append(len(split_to_tokens(sent,True)))

print('(Source) Sentence mean length: ', np.mean(source_len))
print('(Source) Sentence stddev length: ', np.std(source_len))

(Source) Sentence mean length:  26.244692
(Source) Sentence stddev length:  13.854376414156501


In [9]:
# Let us first look at some statistics of the sentences
# Train - target data
target_len = []
for sent in target_sent:
    target_len.append(len(split_to_tokens(sent,False)))

print('(Target) Sentence mean length: ', np.mean(target_len))
print('(Target) Sentence stddev length: ', np.std(target_len))

(Target) Sentence mean length:  28.275308
(Target) Sentence stddev length:  14.925498769057468


In [10]:
# Let us first look at some statistics of the sentences
# Test - source data
test_source_len = []
for sent in test_source_sent:
    test_source_len.append(len(split_to_tokens(sent, True)))
    
print('(Test-Source) Sentence mean length: ', np.mean(test_source_len))
print('(Test-Source) Sentence stddev length: ', np.std(test_source_len))

(Test-Source) Sentence mean length:  26.61
(Test-Source) Sentence stddev length:  14.800604717375572


In [11]:
# Let us first look at some statistics of the sentences
# Test - target data
test_target_len = []
test_tgt_mean, test_tgt_std = 0,0
for sent in test_target_sent:
    test_target_len.append(len(split_to_tokens(sent, False)))
    
print('(Test-Target) Sentence mean length: ', np.mean(test_target_len))
print('(Test-Target) Sentence stddev length: ', np.std(test_target_len))

(Test-Target) Sentence mean length:  29.08
(Test-Target) Sentence stddev length:  16.19424589167399


### Making training and testing data fixed length
Here we get all the source sentences and target sentences to a fixed length. This is, so that we can process the sentences as batches.

In [12]:
# Processing training data

src_unk_count, tgt_unk_count = 0, 0

train_inputs = []
train_outputs = []

# Chosen based on previously found statistics
src_max_sent_length = 41 
tgt_max_sent_length = 61

print('Processing Training Data ...\n')
for s_i, (src_sent, tgt_sent) in enumerate(zip(source_sent,target_sent)):
    # Break source and target sentences to word lists
    src_sent_tokens = split_to_tokens(src_sent,True)
    tgt_sent_tokens = split_to_tokens(tgt_sent,False)
    
    # Append <s> token's ID to the beggining of source sentence
    num_src_sent = [src_dictionary['<s>']]
    # Add the rest of word IDs for words found in the source sentence 
    for tok in src_sent_tokens:
        if tok in src_dictionary.keys():
            num_src_sent.append(src_dictionary[tok])

    # If the lenghth of the source sentence below the maximum allowed length
    # append </s> token's ID to the end
    if len(num_src_sent)<src_max_sent_length:
        num_src_sent.extend([src_dictionary['</s>'] for _ in range(src_max_sent_length - len(num_src_sent))])

    # If the length exceed the maximum allowed length
    # truncate the sentence
    elif len(num_src_sent)>src_max_sent_length:
        num_src_sent = num_src_sent[:src_max_sent_length]
        
    # Make sure the sentence is of length src_max_sent_length
    assert len(num_src_sent)==src_max_sent_length,len(num_src_sent)

    train_inputs.append(num_src_sent)
    
    # Create the numeric target sentence with word IDs
    # append <s> to the beginning and append actual words later
    num_tgt_sent = [tgt_dictionary['<s>']]
    for tok in tgt_sent_tokens:
        if tok in tgt_dictionary.keys():
            num_tgt_sent.append(tgt_dictionary[tok])
        
    ## Modifying the outputs such that all the outputs have max_length elements
    if len(num_tgt_sent)<tgt_max_sent_length:
        num_tgt_sent.extend([tgt_dictionary['</s>'] for _ in range(tgt_max_sent_length - len(num_tgt_sent))])
    elif len(num_tgt_sent)>tgt_max_sent_length:
        num_tgt_sent = num_tgt_sent[:tgt_max_sent_length]
        
    train_outputs.append(num_tgt_sent)
    
print('Unk counts Src: %d, Tgt: %d'%(src_unk_count, tgt_unk_count))
print('Sentences ',len(train_inputs))

assert len(train_inputs)  == len(source_sent),\
        'Size of total elements: %d, Total sentences: %d'\
                %(len(train_inputs),len(source_sent))

# Making inputs and outputs NumPy arrays
train_inputs = np.array(train_inputs, dtype=np.int32)
train_outputs = np.array(train_outputs, dtype=np.int32)

# Make sure number of inputs and outputs dividable by 100
train_inputs = train_inputs[:(train_inputs.shape[0]//100)*100,:]
train_outputs = train_outputs[:(train_outputs.shape[0]//100)*100,:]
print('\t Done processing training data \n')

# Printing some data
print('Samples from training data')
for ti in range(10):
    print('\t',[src_reverse_dictionary[w]  for w in train_inputs[ti,:].tolist()])
    print('\t',[tgt_reverse_dictionary[w]  for w in train_outputs[ti,:].tolist()])
print()
print('\tSentences ',train_inputs.shape[0])

Processing Training Data ...

Unk counts Src: 464223, Tgt: 214783
Sentences  250000
	 Done processing training data 

Samples from training data
	 ['<s>', 'Hier', 'erfahren', 'Sie', ',', 'wie', 'Sie', 'Creative', 'Suite', '2', 'und', 'Creative', 'Suite', '3', 'am', 'besten', 'zusammen', 'mit', 'QuarkXPress', 'nutzen', 'können', '.', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>']
	 ['<s>', 'Here', ',', 'you', '’', 'll', 'find', 'out', 'how', 'Creative', 'Suite', 'users', 'can', 'get', 'the', 'best', 'possible', 'interaction', 'with', 'QuarkXPress', '.', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>']
	 ['<s>', 'Sie',

In [13]:
# Processing Test data

src_unk_count, tgt_unk_count = 0, 0
print('Processing testing data ....\n')
test_inputs = []
test_outputs = []
for s_i, (src_sent,tgt_sent) in enumerate(zip(test_source_sent,test_target_sent)):
    src_sent_tokens = split_to_tokens(src_sent,True)
    tgt_sent_tokens = split_to_tokens(tgt_sent,False)
    
    num_src_sent = [src_dictionary['<s>']]
    for tok in src_sent_tokens:
        if tok in src_dictionary.keys():
            num_src_sent.append(src_dictionary[tok])
    
    num_tgt_sent = [src_dictionary['<s>']]
    for tok in tgt_sent_tokens:
        if tok in tgt_dictionary.keys():
            num_tgt_sent.append(tgt_dictionary[tok])
        
    # Append </s> if the length is not src_max_sent_length
    if len(num_src_sent)<src_max_sent_length:
        num_src_sent.extend([src_dictionary['</s>'] for _ in range(src_max_sent_length - len(num_src_sent))])
    # Truncate the sentence if length is over src_max_sent_length
    elif len(num_src_sent)>src_max_sent_length:
        num_src_sent = num_src_sent[:src_max_sent_length]
        
    assert len(num_src_sent)==src_max_sent_length, len(num_src_sent)

    test_inputs.append(num_src_sent)
    
    # Append </s> is length is not tgt_max_sent_length
    if len(num_tgt_sent)<tgt_max_sent_length:
        num_tgt_sent.extend([tgt_dictionary['</s>'] for _ in range(tgt_max_sent_length - len(num_tgt_sent))])
    # Truncate the sentence if length over tgt_max_sent_length
    elif len(num_tgt_sent)>tgt_max_sent_length:
        num_tgt_sent = num_tgt_sent[:tgt_max_sent_length]
        
    assert len(num_tgt_sent)==tgt_max_sent_length, len(num_tgt_sent)

    test_outputs.append(num_tgt_sent)

# Printing some data
print('Unk counts Tgt: %d, Tgt: %d'%(src_unk_count, tgt_unk_count))    
print('Done processing testing data ....\n')
test_inputs = np.array(test_inputs,dtype=np.int32)
test_outputs = np.array(test_outputs,dtype=np.int32)
print('Samples from training data')
for ti in range(10):
    print('\t',[src_reverse_dictionary[w]  for w in test_inputs[ti,:].tolist()])
    print('\t',[tgt_reverse_dictionary[w]  for w in test_outputs[ti,:].tolist()])

Processing testing data ....

Unk counts Tgt: 212, Tgt: 107
Done processing testing data ....

Samples from training data
	 ['<s>', 'Heute', 'verstehen', 'sich', 'QuarkXPress', '®', '8', ',', 'Photoshop', '®', 'und', 'Illustrator', '®', 'besser', 'als', 'jemals', 'zuvor', '.', 'Dank', 'HTML', 'und', 'CSS', '\xad', 'können', 'Anwender', 'von', 'QuarkXPress', 'inzwischen', 'alle', 'Medien', 'bedienen', ',', 'und', 'das', 'unabhängig', 'von', 'Anwendungen', 'der', 'Adobe', '®', 'Creative']
	 ['<s>', 'Today', ',', 'QuarkXPress', '®', '8', 'has', 'tighter', 'integration', 'with', 'Photoshop', '®', 'and', 'Illustrator', '®', 'than', 'ever', 'before', ',', 'and', 'through', 'standards', 'like', 'HTML', 'and', 'CSS', ',', 'QuarkXPress', 'users', 'can', 'publish', 'across', 'media', 'both', 'independently', 'and', 'alongside', 'Adobe', '®', 'Creative', 'Suite', '®', 'applications', 'like', 'Adobe', 'Flash', '®', '(', 'SWF', ')', 'and', 'Adobe', 'Dreamweaver', '®', '.', '</s>', '</s>', '</s>', '

### Learning word embeddings


#### Run algorithm

In [14]:
# Total number of sentences
tot_sentences = train_inputs.shape[0]
print('Total number of training sentences: ',tot_sentences)

# we keep a cursor for each sentence in the training set
sentence_cursors = [0 for _ in range(tot_sentences)] 

batch_size = 64
embedding_size = 128 # Dimension of the embedding vector.

# Defining various things needed by the python script
define_data_and_hyperparameters(
    tot_sentences, 
    src_max_sent_length, 
    tgt_max_sent_length, 
    src_dictionary, 
    tgt_dictionary,
    src_reverse_dictionary, 
    tgt_reverse_dictionary, 
    train_inputs, 
    train_outputs, 
    embedding_size,
    vocabulary_size
)

# Print some batches to make sure the data generator is correct
print_some_batches()

Total number of training sentences:  250000

with window_size = 1:
    batch: [['<s>', 'Ende'], ['<s>', ':'], ['<s>', 'Sie'], ['<s>', 'AB'], ['<s>', 'gibt'], ['<s>', 'hat'], ['<s>', 'nimmt'], ['<s>', 'möchten']]
    labels: ['Seit', 'F', 'Denken', 'Der', 'Es', 'Jaca', 'Man', 'Wann']

with window_size = 2:
    batch: [['<s>', 'Betten', '13', '€'], ['<s>', '##STAR##', 'Programm', 'teilt'], ['<s>', 'Wir', 'diese', '<unk>'], ['<s>', 'Der', 'zwischen', '<unk>'], ['<s>', 'Schließlich', 'die', 'Zivilgesellschaft'], ['<s>', '<unk>', ',', '<unk>'], ['<s>', 'Das', 'besitzt', 'großzügig'], ['<s>', 'Unseren', 'steht', 'auch']]
    labels: ['ab', 'Das', 'haben', 'Wanderweg', 'muss', 'Garten', 'Appartement', 'Gäste']


In [15]:
# Define TensorFlow ops for learning word embeddings
define_word2vec_tensorflow(batch_size)

Defining 4 embedding lookups representing each word in the context
Stacked embedding size: [64, 128, 4]
Reduced mean embedding size: [64, 128]


In [16]:
# Run embedding learning for source language
# Stores the de-embeddings-tmp.npy into the disk
run_word2vec_source(batch_size)
# Run embedding learning for target language
# Stores the en-embeddings-tmp.npy to the disk
run_word2vec_target(batch_size)

Initialized
Average loss at step 2000: 3.532927
Average loss at step 4000: 2.701178
Average loss at step 6000: 2.484155
Average loss at step 8000: 2.377980
Average loss at step 10000: 2.294842
Nearest to und: unterhält, Schenken, bisher, besinnen, Güterverkehrs, Autofokus, Tennis, Ruhige,
Nearest to bei: Sinn, Laptop, hielten, Klimagipfel, Setzen, siegen, Yeshé, Laufe,
Nearest to ich: Ich, Uniform, bildete, dies, beispielsweise, mysqld, down, kürzere,
Nearest to -: Seltenheit, desk, 1790, Gesetzesvorlage, Verträge, förderlich, letztendlichen, Swisscom,
Nearest to wenn: Wahrlich, sage, Siehe, dein, D3, weiter, daß, siehe,
Nearest to Union: Altstadt, much, Fläche, quite, Benalmadena, Zusammenstellung, hundertprozentig, Frankfurt,
Nearest to die: diese, formiert, Diese, Dichter, BIS, ihre, Ihre, Ihren,
Nearest to Die: Unsere, die, Eine, der, Ferien, vegetarischen, Seine, jahre,
Nearest to :: &#93;, página, &gt;, Kay, sicherheitsrelevanten, vous, gestiegenen, IUU,
Nearest to alle: Mitglied

### Flipping the Input Data
Changin the order of the sentence of the target language improves the performance of NMT systems. Because when reversed, it helps the NMT system to establish a strong connection as the last word of the source language and the last word of the target language will be closest to each other. DON'T RUN THIS MULTIPLE TIMES as running two times gives original.


In [17]:
## Reverse the Germen sentences
# Remember reversing the source sentence gives better performance
# DON'T RUN THIS MULTIPLE TIMES as running two times gives original
train_inputs = np.fliplr(train_inputs)
test_inputs = np.fliplr(test_inputs)

print('Training and Test source data after flipping ')
print('\t',[src_reverse_dictionary[w] for w in train_inputs[0,:].tolist()])
print('\t',[tgt_reverse_dictionary[w] for w in test_inputs[0,:].tolist()])
print()
print('\t',[src_reverse_dictionary[w] for w in train_inputs[10,:].tolist()])
print('\t',[tgt_reverse_dictionary[w] for w in test_inputs[10,:].tolist()])

print()
print('\nTesting data after flipping')
print('\t',[src_reverse_dictionary[w] for w in test_inputs[0,:].tolist()])

Training and Test source data after flipping 
	 ['</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '.', 'können', 'nutzen', 'QuarkXPress', 'mit', 'zusammen', 'besten', 'am', '3', 'Suite', 'Creative', 'und', '2', 'Suite', 'Creative', 'Sie', 'wie', ',', 'Sie', 'erfahren', 'Hier', '<s>']
	 ['tray', 'road', 'mistakes', 'of', 'expect', 'a', 'tabled', 'with', 'and', 'the', 'posts', 'useful', 'out', 'waiting', 'wounded', 'a', 'drinks', 'been', 'stand', '26th', 'and', 'senior', 'personal', ',', 'difficulties', 'qualifications', 'an', 'rather', 'road', 'rewriting', 'and', 'road', 'unsustainable', 'the', '2007', 'road', 'wounded', 'not', 'throughout', 'amendment', '<s>']

	 ['</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '.', ')', 'Import', '##AT##-##AT##', 'PSD', '&gt;', 'Fenster', '(', 'Import', '##AT##-##AT##', 'PSD', 'Palette', 'die', 

## Data Generations for MT

In [18]:
input_size = embedding_size

class DataGeneratorMT(object):
    
    def __init__(self,batch_size,num_unroll,is_source, is_train):
        # Number of data points in a batch
        self._batch_size = batch_size
        # Number of unrollings
        self._num_unroll = num_unroll
        # Cursors for each element in batch
        self._cursor = [0 for offset in range(self._batch_size)]
        
        # Loading the learnt word embeddings
        self._src_word_embeddings = np.load('de-embeddings.npy')
        self._tgt_word_embeddings = np.load('en-embeddings.npy')
        
        # The sentence IDs being currently processed to create the 
        # current batch
        self._sent_ids = None
        
        # We want a batch of data from source or target?
        self._is_source = is_source
        # Is this training or testing data?
        self._is_train = is_train
                
    def next_batch(self, sent_ids):
        
        # Depending on wheter we want source or target data
        # change the maximum sentence length
        if self._is_source:
            max_sent_length = src_max_sent_length
        else:
            max_sent_length = tgt_max_sent_length
            
        # Arrays to hold input and output data
        # Word embeddings (current word)
        batch_data = np.zeros((self._batch_size,input_size),dtype=np.float32)
        # One-hot encoded label (next word)
        batch_labels = np.zeros((self._batch_size,vocabulary_size),dtype=np.float32)
        
        
        # Populate each index of the batch
        for b in range(self._batch_size):
            
            # Sentence IDs to get data from
            sent_id = sent_ids[b]
            
            # If generating data with source sentences
            # use src_word_embeddings
            if self._is_source:
                # Depending on whether we need training data or testind data
                # choose the previously created training or testind data
                if self._is_train:
                    sent_text = train_inputs[sent_id]
                else:
                    sent_text = test_inputs[sent_id]
                             
                # Populate the batch data arrays
                batch_data[b] = self._src_word_embeddings[sent_text[self._cursor[b]],:]
                batch_labels[b] = np.zeros((vocabulary_size),dtype=np.float32)
                batch_labels[b,sent_text[self._cursor[b]+1]] = 1.0
            # If generating data with target sentences
            # use tgt_word_embeddings
            else:
                # Depending on whether we need training data or testind data
                # choose the previously created training or testind data
                if self._is_train:
                    sent_text = train_outputs[sent_id]
                else:
                    sent_text = test_outputs[sent_id]
                
                # We cannot avoid having two different embedding vectors for <s> token
                # in soruce and target languages
                # Therefore, if the symbol appears, we always take the source embedding vector
                if sent_text[self._cursor[b]]!=tgt_dictionary['<s>']:
                    batch_data[b] = self._tgt_word_embeddings[sent_text[self._cursor[b]],:]
                else:
                    batch_data[b] = self._src_word_embeddings[sent_text[self._cursor[b]],:]
                
                # Populate the data arrays
                batch_labels[b] = np.zeros((vocabulary_size),dtype=np.float32)
                batch_labels[b,sent_text[self._cursor[b]+1]] = 1.0
            
            # Update the cursor for each batch index
            self._cursor[b] = (self._cursor[b]+1)%(max_sent_length-1)
             
        return batch_data,batch_labels
        
    def unroll_batches(self,sent_ids):
        
        # Only if new sentence IDs if provided
        # else it will use the previously defined 
        # sent_ids continuously
        if sent_ids is not None:
            
            self._sent_ids = sent_ids
            # Unlike in the previous exercises we do not process a single sequence
            # over many iterations of unrollings. We process either a source sentence or target sentence
            # at a single go. So we reset the _cursor evrytime we generate a batch
            self._cursor = [0 for _ in range(self._batch_size)]
                
        unroll_data,unroll_labels = [],[]
        
        # Unrolling data over time
        for ui in range(self._num_unroll):
            
            if self._is_source:
                data, labels = self.next_batch(self._sent_ids)
            else:
                data, labels = self.next_batch(self._sent_ids)
                    
            unroll_data.append(data)
            unroll_labels.append(labels)
        
        # Return unrolled data and sentence IDs
        return unroll_data, unroll_labels, self._sent_ids
    
    def reset_indices(self):
        self._cursor = [0 for offset in range(self._batch_size)]

In [19]:
# Running a tiny set to see if the implementation correct
dg = DataGeneratorMT(batch_size=5,num_unroll=20,is_source=True, is_train=True)
u_data, u_labels, _ = dg.unroll_batches([0,1,2,3,4])

print('Source data')
for _, lbl in zip(u_data,u_labels):
    # the the string words for returned word IDs and display the results
    print([src_reverse_dictionary[w] for w in np.argmax(lbl,axis=1).tolist()])

Source data
['</s>', '</s>', '</s>', '</s>', '</s>']
['</s>', '</s>', '</s>', '</s>', '</s>']
['</s>', '</s>', '</s>', '</s>', '</s>']
['</s>', '</s>', '</s>', '</s>', '</s>']
['</s>', '</s>', '</s>', '</s>', '</s>']
['</s>', '</s>', '</s>', '</s>', '</s>']
['</s>', '</s>', '</s>', '</s>', '</s>']
['</s>', '</s>', '</s>', '</s>', '</s>']
['</s>', '</s>', '</s>', '</s>', '</s>']
['</s>', '</s>', '</s>', '</s>', '</s>']
['</s>', '</s>', '</s>', '</s>', '</s>']
['</s>', '</s>', '.', '</s>', '</s>']
['</s>', '</s>', 'bietet', '.', '</s>']
['</s>', '</s>', 'Dateiformat', 'nutzen', '</s>']
['</s>', '</s>', '##AT##-##AT##', 'optimal', '</s>']
['</s>', '</s>', 'PSD', 'Bilder', '</s>']
['</s>', '</s>', 'das', 'Ihre', '.']
['</s>', '</s>', 'über', 'für', 'werden']
['.', '</s>', 'Photoshop', 'es', 'ausgewählt']
['können', '.', 'mit', 'Sie', 'Verwendungszweck']


In [20]:
# Running a tiny set to see if the implementation correct
dg = DataGeneratorMT(batch_size=5,num_unroll=30,is_source=False, is_train=True)
u_data, u_labels, _ = dg.unroll_batches([0,2,3,4,5])
print('\nTarget data batch')
for d_i,(_, lbl) in enumerate(zip(u_data,u_labels)):
    # the the string words for returned word IDs and display the results
    print([tgt_reverse_dictionary[w] for w in np.argmax(lbl,axis=1).tolist()])


Target data batch
['Here', 'QuarkXPress', 'In', 'For', 'If']
[',', '8', 'this', 'example', 'you']
['you', 'is', 'section', ',', 'use']
['’', 'considered', 'we', 'you', 'PSD']
['ll', 'by', '’', 'may', ',']
['find', 'many', 'll', 'have', 'you']
['out', 'to', 'explain', 'multiple', 'can']
['how', 'have', 'when', 'layers', 'switch']
['Creative', 'the', 'you', 'in', 'those']
['Suite', 'best', 'should', 'your', 'layers']
['users', 'integration', 'use', 'PSD', 'on']
['can', 'with', 'the', 'with', 'or']
['get', 'Photoshop', 'PSD', 'different', 'off']
['the', '’', 'format', 'product', 'in']
['best', 's', 'for', 'shots', 'QuarkXPress']
['possible', 'PSD', 'your', ',', 'without']
['interaction', 'file', 'images', 'which', 'having']
['with', 'format', 'and', 'will', 'to']
['QuarkXPress', 'of', 'how', 'vary', 'save']
['.', 'any', 'to', 'from', 'a']
['</s>', 'layout', 'get', 'publication', 'separate']
['</s>', 'tool', 'the', 'to', 'TIFF']
['</s>', 'available', 'most', 'publication', 'for']
['</s>',

# NMT

## Basics

### Hyperparameters

In [21]:
tgt_emb_mat = np.load('en-embeddings.npy')
input_size = tgt_emb_mat.shape[1]

num_nodes = 128
batch_size = 10

# We unroll the full length at one go
# both source and target sentences
enc_num_unrollings = 40
dec_num_unrollings = 60

### Defining Input/Output Placeholders

In [22]:
tf.reset_default_graph()

# Target embeddings are needed to compute the test outputs
tgt_word_embeddings = tf.convert_to_tensor(tgt_emb_mat,name='tgt_embeddings')

In [23]:
print('Defining encoder data placeholders')

# Training Input placeholders (Encoder)
# Encoder related input data, we directly feed in the embeddings
enc_train_inputs = []

# Defining unrolled training inputs
for ui in range(enc_num_unrollings):
    enc_train_inputs.append(tf.placeholder(tf.float32, shape=[batch_size,input_size],name='train_inputs_%d'%ui))

Defining encoder data placeholders


In [25]:
print('Defining decoder data placeholders')
# Training Input/Output/Mask data (Decoder)
# Decoder inputs and outputs
dec_train_inputs, dec_train_labels = [],[]
# We use masking to mask out the any of the </s> elements
# from the loss computation in the decoder
dec_train_masks = []

# Defining unrolled training inputs
for ui in range(dec_num_unrollings):
    dec_train_inputs.append(tf.placeholder(tf.float32, shape=[batch_size,input_size],name='dec_train_inputs_%d'%ui))
    dec_train_labels.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size], name = 'dec_train_labels_%d'%ui))
    dec_train_masks.append(tf.placeholder(tf.float32, shape=[batch_size,1],name='dec_train_masks_%d'%ui))

    
enc_test_input = [tf.placeholder(tf.float32, shape=[batch_size,input_size], name='test_input_%d'%ui) for ui in range(enc_num_unrollings)]
dec_test_input = tf.nn.embedding_lookup(tgt_word_embeddings,[tgt_dictionary['<s>']])
print('Done')

Defining decoder data placeholders
Done


### Encoder Model

Got a `tf.compat.v1` has no module `contrib` error. Solution was to copy paste this file that has the `xavier_initializer()`

https://github.com/tensorflow/tensorflow/blob/590d6eef7e91a6a7392c8ffffb7b58f2e0c8bc6b/tensorflow/contrib/layers/python/layers/initializers.py

In [32]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import math

from tensorflow.python.framework import dtypes
from tensorflow.python.ops import random_ops


def xavier_initializer(uniform=True, seed=None, dtype=dtypes.float32):
  """Returns an initializer performing "Xavier" initialization for weights.
  This function implements the weight initialization from:
  Xavier Glorot and Yoshua Bengio (2010):
           [Understanding the difficulty of training deep feedforward neural
           networks. International conference on artificial intelligence and
           statistics.](
           http://www.jmlr.org/proceedings/papers/v9/glorot10a/glorot10a.pdf)
  This initializer is designed to keep the scale of the gradients roughly the
  same in all layers. In uniform distribution this ends up being the range:
  `x = sqrt(6. / (in + out)); [-x, x]` and for normal distribution a standard
  deviation of `sqrt(2. / (in + out))` is used.
  Args:
    uniform: Whether to use uniform or normal distributed random initialization.
    seed: A Python integer. Used to create random seeds. See
          `tf.compat.v1.set_random_seed` for behavior.
    dtype: The data type. Only floating point types are supported.
  Returns:
    An initializer for a weight matrix.
  """
  return variance_scaling_initializer(factor=1.0, mode='FAN_AVG',
                                      uniform=uniform, seed=seed, dtype=dtype)

xavier_initializer_conv2d = xavier_initializer


def variance_scaling_initializer(factor=2.0, mode='FAN_IN', uniform=False,
                                 seed=None, dtype=dtypes.float32):
  """Returns an initializer that generates tensors without scaling variance.
  When initializing a deep network, it is in principle advantageous to keep
  the scale of the input variance constant, so it does not explode or diminish
  by reaching the final layer. This initializer use the following formula:
  ```python
    if mode='FAN_IN': # Count only number of input connections.
      n = fan_in
    elif mode='FAN_OUT': # Count only number of output connections.
      n = fan_out
    elif mode='FAN_AVG': # Average number of inputs and output connections.
      n = (fan_in + fan_out)/2.0
      truncated_normal(shape, 0.0, stddev=sqrt(factor / n))
  ```
  * To get [Delving Deep into Rectifiers](
     http://arxiv.org/pdf/1502.01852v1.pdf) (also know as the "MSRA 
     initialization"), use (Default):<br/>
    `factor=2.0 mode='FAN_IN' uniform=False`
  * To get [Convolutional Architecture for Fast Feature Embedding](
     http://arxiv.org/abs/1408.5093), use:<br/>
    `factor=1.0 mode='FAN_IN' uniform=True`
  * To get [Understanding the difficulty of training deep feedforward neural
    networks](http://jmlr.org/proceedings/papers/v9/glorot10a/glorot10a.pdf),
    use:<br/>
    `factor=1.0 mode='FAN_AVG' uniform=True.`
  * To get `xavier_initializer` use either:<br/>
    `factor=1.0 mode='FAN_AVG' uniform=True`, or<br/>
    `factor=1.0 mode='FAN_AVG' uniform=False`.
  Args:
    factor: Float.  A multiplicative factor.
    mode: String.  'FAN_IN', 'FAN_OUT', 'FAN_AVG'.
    uniform: Whether to use uniform or normal distributed random initialization.
    seed: A Python integer. Used to create random seeds. See
          `tf.compat.v1.set_random_seed` for behavior.
    dtype: The data type. Only floating point types are supported.
  Returns:
    An initializer that generates tensors with unit variance.
  Raises:
    ValueError: if `dtype` is not a floating point type.
    TypeError: if `mode` is not in ['FAN_IN', 'FAN_OUT', 'FAN_AVG'].
  """
  if not dtype.is_floating:
    raise TypeError('Cannot create initializer for non-floating point type.')
  if mode not in ['FAN_IN', 'FAN_OUT', 'FAN_AVG']:
    raise TypeError('Unknown mode %s [FAN_IN, FAN_OUT, FAN_AVG]', mode)

  # pylint: disable=unused-argument
  def _initializer(shape, dtype=dtype, partition_info=None):
    """Initializer function."""
    if not dtype.is_floating:
      raise TypeError('Cannot create initializer for non-floating point type.')
    # Estimating fan_in and fan_out is not possible to do perfectly, but we try.
    # This is the right thing for matrix multiply and convolutions.
    if shape:
      fan_in = float(shape[-2]) if len(shape) > 1 else float(shape[-1])
      fan_out = float(shape[-1])
    else:
      fan_in = 1.0
      fan_out = 1.0
    for dim in shape[:-2]:
      fan_in *= float(dim)
      fan_out *= float(dim)
    if mode == 'FAN_IN':
      # Count only number of input connections.
      n = fan_in
    elif mode == 'FAN_OUT':
      # Count only number of output connections.
      n = fan_out
    elif mode == 'FAN_AVG':
      # Average number of inputs and output connections.
      n = (fan_in + fan_out) / 2.0
    if uniform:
      # To get stddev = math.sqrt(factor / n) need to adjust for uniform.
      limit = math.sqrt(3.0 * factor / n)
      return random_ops.random_uniform(shape, -limit, limit,
                                       dtype, seed=seed)
    else:
      # To get stddev = math.sqrt(factor / n) need to adjust for truncated.
      trunc_stddev = math.sqrt(1.3 * factor / n)
      return random_ops.truncated_normal(shape, 0.0, trunc_stddev, dtype,
                                         seed=seed)
  # pylint: enable=unused-argument

  return _initializer

In [33]:
print('Defining Encoder Parameters')
with tf.variable_scope('Encoder'):
    
    # Input gate (i_t) - How much memory to write to cell state
    # We use xavier intialization as this gives better results 
    enc_ix = tf.get_variable('ix',shape=[input_size, num_nodes],
                             initializer = xavier_initializer())
    enc_im = tf.get_variable('im',shape=[num_nodes, num_nodes],
                             initializer = xavier_initializer())
    enc_ib = tf.Variable(tf.random_uniform([1, num_nodes],-0.05, 0.05),name='ib')
    
    # Forget gate (f_t) - How much memory to discard from cell state
    enc_fx = tf.get_variable('fx',shape=[input_size, num_nodes],
                             initializer = xavier_initializer())
    enc_fm = tf.get_variable('fm',shape=[num_nodes, num_nodes],
                             initializer = xavier_initializer())
    enc_fb = tf.Variable(tf.random_uniform([1, num_nodes],-0.05, 0.05),name='fb')
    
    # Candidate value (c~_t) - Used to compute the current cell state                            
    enc_cx = tf.get_variable('cx',shape=[input_size, num_nodes],
                             initializer = xavier_initializer())
    enc_cm = tf.get_variable('cm',shape=[num_nodes, num_nodes],
                             initializer = xavier_initializer())
    enc_cb = tf.Variable(tf.random_uniform([1, num_nodes],-0.05,0.05),name='cb') 
    
    # Output gate - How much memory to output from the cell state
    enc_ox = tf.get_variable('ox',shape=[input_size, num_nodes],
                             initializer = xavier_initializer())
    enc_om = tf.get_variable('om',shape=[num_nodes, num_nodes],
                             initializer = xavier_initializer())
    enc_ob = tf.Variable(tf.random_uniform([1, num_nodes],-0.05,0.05),name='ob') 
    
    # Variables saving state across unrollings (testing).
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False, name='train_output')
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False, name = 'train_cell')
    
    saved_test_output = tf.Variable(tf.zeros([batch_size, num_nodes]),trainable=False, name='test_output')
    saved_test_state = tf.Variable(tf.zeros([batch_size, num_nodes]),trainable=False, name='test_cell')

print('Done')

Defining Encoder Parameters
Done


### Decoder: LSTM + Softmax

In [34]:
print('Defining Decoder Parameters')
with tf.variable_scope('Decoder'):
    
    # Input gate (i_t) - How much memory to write to cell state
    # We use xavier intialization as this gives better results
    dec_ix = tf.get_variable('ix',shape=[input_size, num_nodes],
                             initializer = xavier_initializer())
    dec_im = tf.get_variable('im',shape=[num_nodes, num_nodes],
                             initializer = xavier_initializer())
    dec_ib = tf.Variable(tf.random_uniform([1, num_nodes],-0.05, 0.05),name='ib')    
    
    # Forget gate (f_t) - How much memory to discard from cell state
    dec_fx = tf.get_variable('fx',shape=[input_size, num_nodes],
                             initializer = xavier_initializer())
    dec_fm = tf.get_variable('fm',shape=[num_nodes, num_nodes],
                             initializer = xavier_initializer())
    dec_fb = tf.Variable(tf.random_uniform([1, num_nodes],-0.05, 0.05),name='fb')    
    
    # Candidate value (c~_t) - Used to compute the current cell state                             
    dec_cx = tf.get_variable('cx',shape=[input_size, num_nodes],
                             initializer = xavier_initializer())
    dec_cm = tf.get_variable('cm',shape=[num_nodes, num_nodes],
                             initializer = xavier_initializer())
    dec_cb = tf.Variable(tf.random_uniform([1, num_nodes],-0.05,0.05),name='cb')     
    
    # Output gate - How much memory to output from the cell state
    dec_ox = tf.get_variable('ox',shape=[input_size, num_nodes],
                             initializer = xavier_initializer())
    dec_om = tf.get_variable('om',shape=[num_nodes, num_nodes],
                             initializer = xavier_initializer())
    dec_ob = tf.Variable(tf.random_uniform([1, num_nodes],-0.05,0.05),name='ob') 
    
    # Softmax Classifier weights and biases.
    # If we are using sampled softmax loss, the weights dims shouldbe [50000, 64]
    # If not, then [64, 50000]
    w = tf.get_variable('softmax_weights',shape=[num_nodes, vocabulary_size], 
                        initializer = xavier_initializer())
    b = tf.Variable(tf.random_uniform([vocabulary_size],-0.05,-0.05),name='softmax_bias')

print('Done')

Defining Decoder Parameters
	Done


### Defining LSTM Computations
Here we first define two function enc_lstm_cell and dec_lstm_cell which define the LSTM cell computations. 

Next we define the computations to compute the final state variables of the encoder, feeding that into the decoder as the intial state and finally computing the LSTM output, logit values and the predictions.

In [35]:
# Definition of the cell computation (Encoder)
def enc_lstm_cell(i, o, state):
    """Create a LSTM cell"""
    input_gate = tf.sigmoid(tf.matmul(i, enc_ix) + tf.matmul(o, enc_im) + enc_ib)
    forget_gate = tf.sigmoid(tf.matmul(i, enc_fx) + tf.matmul(o, enc_fm) + enc_fb)
    update = tf.matmul(i, enc_cx) + tf.matmul(o, enc_cm) + enc_cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, enc_ox) + tf.matmul(o, enc_om) + enc_ob)
    return output_gate * tf.tanh(state), state

In [36]:
# Definition of the cell computation (Decoder)
def dec_lstm_cell(i, o, state):
    """Create a LSTM cell"""
    input_gate = tf.sigmoid(tf.matmul(i, dec_ix) + tf.matmul(o, dec_im) + dec_ib)
    forget_gate = tf.sigmoid(tf.matmul(i, dec_fx) + tf.matmul(o, dec_fm) + dec_fb)
    update = tf.matmul(i, dec_cx) + tf.matmul(o, dec_cm) + dec_cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, dec_ox) + tf.matmul(o, dec_om) + dec_ob)
    return output_gate * tf.tanh(state), state

In [37]:
# Training inference logic
# Holds the outputs of the all unrolled LSTM steps
outputs = list()

In [38]:
# Initialize the output and state variables
# with zeros and iteratively update these two 
# variables with the LSTM's output and state
output = saved_output
state = saved_state

In [40]:
print('Calculating Encoder Output')
# Update the output and state of the encoder iteratively
for i in enc_train_inputs:
    output, state = enc_lstm_cell(i, output,state)

Calculating Encoder Output


In [41]:
print('Calculating Decoder Output')
# With the computations of the enc_lstm_cell done,
# calculate the output and state of the decoder
with tf.control_dependencies([saved_output.assign(output),
                             saved_state.assign(state)]):
    # Calculate the decoder state and output iteratively
    for i in dec_train_inputs:
        output, state = dec_lstm_cell(i, output, state)
        outputs.append(output)

Calculating Decoder Output


In [42]:
# Calculate the logits of the decoder for all unrolled steps
logits = tf.matmul(tf.concat(axis=0, values=outputs), w) + b

In [43]:
# Predictions from the decoder
train_prediction = tf.nn.softmax(logits)

In [44]:
# Testing related inference logic
# We calculate the test predictions for the maximum 
# num_unrollings allowed for target language
test_output  = saved_test_output
test_state = saved_test_state

In [45]:
test_predictions = []

In [46]:
# Compute the encoder output iteratively
for i in enc_test_input:
    test_output, test_state = enc_lstm_cell(i, test_output,test_state)

In [47]:
# Compute the decoder output iteratively
# where the input is whatever the previously output word's embedding
with tf.control_dependencies([saved_test_output.assign(test_output),
                                 saved_test_state.assign(test_state)]):
    for i in range(dec_num_unrollings):

        test_output, test_state = dec_lstm_cell(dec_test_input, test_output, test_state)

        test_prediction = tf.nn.softmax(tf.nn.xw_plus_b(test_output, w, b))
        
        dec_test_input = tf.nn.embedding_lookup(tgt_word_embeddings,tf.argmax(test_prediction,axis=1))
        test_predictions.append(tf.argmax(test_prediction,axis=1))

### Calculating the Loss
Here we calculate the loss. Loss is calculated by summing all the losses obtained across the time axis and averagin over the batch axis.

In [48]:
print('Calculating Loss')

# No need for a tf.contro_dependencies(...) clause here
# Because we restart the state anyway after each sentence batch
loss_batch = tf.concat(
    axis=0,
    values=dec_train_masks
    ) * tf.nn.softmax_cross_entropy_with_logits_v2(
        logits=logits, 
        labels=tf.concat(axis=0, values=dec_train_labels)
    )
loss = tf.reduce_mean(loss_batch)

Calculating Loss


### Optimizer
We define the model optimization specific operations. We use two optimizers here; Adam and SGD. I observed that using Adam only cause the model to exhibit some undesired behaviors in the long run. Therefore we use Adam to get a good initial estimate for the SGD and use SGD from that point onwards.

In [49]:
print('Defining Optimizers')
# These are used to decay learning rate over time
global_step = tf.Variable(0, trainable=False)
inc_gstep = tf.assign(global_step,global_step + 1)

Defining Optimizers


In [50]:
# We use two optimizers, when the optimizer changes
# we reset the global step
reset_gstep = tf.assign(global_step,0)

In [51]:
# Calculate decaying learning rate
learning_rate = tf.maximum(
    tf.train.exponential_decay(
        0.005, global_step, decay_steps=1, decay_rate=0.95, staircase=True
    ), 0.00001)

sgd_learning_rate = tf.maximum(
    tf.train.exponential_decay(
        0.005, global_step, decay_steps=1, decay_rate=0.95, staircase=True
    ), 0.00001)

In [52]:
# We use two optimizers: Adam and naive SGD
# using Adam in the long run produced undesirable results 
# (e.g.) sudden fluctuations in BLEU
# Therefore we use Adam to get a good starting point for optimizing
# and then switch to SGD from that point onwards
with tf.variable_scope('Adam'):
    optimizer = tf.train.AdamOptimizer(learning_rate)
with tf.variable_scope('SGD'):
    sgd_optimizer = tf.train.GradientDescentOptimizer(sgd_learning_rate)

In [53]:
# Calculates gradients with clipping for Adam
gradients, v = zip(*optimizer.compute_gradients(loss))
gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
optimize = optimizer.apply_gradients(zip(gradients, v))

In [54]:
# Calculates gradients with clipping for SGD
sgd_gradients, v = zip(*sgd_optimizer.compute_gradients(loss))
sgd_gradients, _ = tf.clip_by_global_norm(sgd_gradients, 5.0)
sgd_optimize = optimizer.apply_gradients(zip(sgd_gradients, v))

In [56]:
# Make sure gradients exist flowing from decoder to encoder
for (g_i,v_i) in zip(gradients,v):
    assert g_i is not None, 'Gradient none for %s'%(v_i.name)

### Resetting Train and Test States

In [57]:
# Reset training state
reset_train_state = tf.group(
    tf.assign(saved_output, tf.zeros([batch_size, num_nodes])),
    tf.assign(saved_state, tf.zeros([batch_size, num_nodes]))
)

reset_test_state = tf.group(
    saved_test_output.assign(tf.zeros([batch_size, num_nodes])),
    saved_test_state.assign(tf.zeros([batch_size, num_nodes]))
)

### Functions for Evaulating and Printing Results

In [58]:
def print_and_save_train_predictions(du_labels, tr_pred, rand_idx, train_prediction_text_fname):
    '''
    Use this to print some predicted training samples and save it to file
    du_labels: Decoder's unrolled labels (this is a list of dec_num_unrollings 
    where each item is [batch_size, vocabulary_size])
    tr_pred: This is an array [dec_num_unrollings*batch_size, vocabulary_size] array
    rand_idx: Some random index we use to pick a data point to print
    train_prediction_text_fname: The file we save the prediction results into
    '''

    # This print_str will be written to the text file as well as printed here
    print_str = 'Actual: ' 
    
    # We can get each label corresponding to some sentence by traversing the
    # concatenated labels array ([dec_num_unrollings*batch_size, vocabulary_size])
    # with a batch_size stride
    for w in np.argmax(np.concatenate(du_labels,axis=0)[rand_idx::batch_size],axis=1).tolist():
        # Update the print_str
        print_str += tgt_reverse_dictionary[w] + ' '
        # When we encounter the end of sentence </s> we stop printing
        if tgt_reverse_dictionary[w] == '</s>':
            break
    print(print_str)
    
    # Write to file
    with open(os.path.join(log_dir, train_prediction_text_fname),'a',encoding='utf-8') as fa:                
        fa.write(print_str+'\n')  

    # Now print the predicted data by following the same procedure as above
    print()
    print_str = 'Predicted: '
    for w in np.argmax(tr_pred[rand_idx::batch_size],axis=1).tolist():
        print_str += tgt_reverse_dictionary[w] + ' '
        # When we encounter the end of sentence </s> we stop printing
        if tgt_reverse_dictionary[w] == '</s>':
            break
    print(print_str)
    with open(os.path.join(log_dir, train_prediction_text_fname),'a',encoding='utf-8') as fa:                
        fa.write(print_str+'\n')    
    
    
def print_and_save_test_predictions(test_du_labels, test_pred_unrolled, batch_id, test_rand_idx, test_prediction_text_fname):
    '''
    Use this to print some predicted training samples and save it to file
    test_du_labels: Decoder's unrolled labels (this is a list of dec_num_unrollings 
    where each item is [batch_size, vocabulary_size])
    test_pred_unrolled: This is an array [dec_num_unrollings*batch_size, vocabulary_size] array
    batch_id: We need this to retrieve the actual sentence for the predicted 
    test_rand_idx: Some random index we use to pick a data point to print
    test_prediction_text_fname: The file we save the prediction results into
    '''
    
    # Print the actual sentence
    print('DE: ',test_source_sent[(batch_id*batch_size)+test_rand_idx])
    # print_str is the string we display as results and write to a file
    print_str = '\t EN (TRUE):' + test_target_sent[(batch_id*batch_size)+test_rand_idx]
    print(print_str + '\n')

    # Printing predictions
    print_str = '\t EN (Predicted): ' 
    
    for test_pred in test_pred_unrolled:                            
        print_str += tgt_reverse_dictionary[test_pred[test_rand_idx]] + ' '
        if tgt_reverse_dictionary[test_pred[test_rand_idx]] == '</s>':
            break
    print(print_str + '\n')

    # Write the results to text file
    with open(os.path.join(log_dir, test_prediction_text_fname),'a',encoding='utf-8') as fa:                                
        fa.write(print_str+'\n') 
        
def create_bleu_ref_candidate_lists(all_preds, all_labels):
    '''
    Creates two lists (candidate list and reference list) for calcluating BLEU
    all_preds: All the predictions
    all_labels: Correspondign all the actual labels
    Returns
    cand_list: List (sentences) of lists (words in a sentence)
    ref_list: List (sentences) of lists (words in a sentence)
    '''
    bleu_labels, bleu_preds = [],[]
    
    # calculate bleu score:        
    # We iterate batch_size times as i=0,1,2,...,batch_size while grabbing 
    # i, i+batch_size, i+2*batch_size, i+3*batch_size elements from all_labels and all_preds
    # This because the labels/predicitons belonging to same sentence are interleaved by batch_size 
    # due to the way concatenate labels and predictions
    # Taking elements interleaved by batch_size gives the sequence of words belonging to the same sentence
    ref_list, cand_list = [],[]
    for b_i in range(batch_size):
        tmp_lbl = all_labels[b_i::batch_size]            
        tmp_lbl = tmp_lbl[np.where(tmp_lbl != tgt_dictionary['</s>'])]            
        ref_str = ' '.join([tgt_reverse_dictionary[lbl] for lbl in tmp_lbl])
        ref_list.append([ref_str])

        tmp_pred = all_preds[b_i::batch_size]
        tmp_pred = tmp_pred[np.where(tmp_pred != tgt_dictionary['</s>'])]
        cand_str = ' '.join([tgt_reverse_dictionary[pre] for pre in tmp_pred])
        cand_list.append(cand_str)

    return cand_list, ref_list

### Defining a Single Step of Training
We now define a function to train the NMT model for a single step. It takes in encoder inputs, decoder inputs and decoder labels and train the NMT for a single step.

In [59]:
def train_single_step(eu_data, du_data, du_labels):
    '''
    Define a single training step
    eu_data: Unrolled encoder inputs (word embeddings)
    du_data: Unrolled decoder inputs (word embeddings)
    du_labels: Unrolled decoder outputs (one hot encoded words)
    '''
    # Fill the feed dict (Encoder)
    feed_dict = {}
    for ui,dat in enumerate(eu_data):            
        feed_dict[enc_train_inputs[ui]] = dat    
    
    
    # Fill the feed dict (Decoder) 
    for ui,(dat,lbl) in enumerate(zip(du_data,du_labels)):            
        feed_dict[dec_train_inputs[ui]] = dat
        feed_dict[dec_train_labels[ui]] = lbl
        # The mask masks the </s> items from being part of the loss
        d_msk = (np.logical_not(np.argmax(lbl,axis=1)==tgt_dictionary['</s>'])).astype(np.int32).reshape(-1,1)
        feed_dict[dec_train_masks[ui]] = d_msk
    
    # ======================= OPTIMIZATION ==========================
    # Using Adam in long term gives very weird behaviors in loss
    # so after 20000 iterations we change the optimizer to SGD
    if (step+1)<20000:
        _,l,tr_pred = sess.run([optimize,loss,train_prediction], feed_dict=feed_dict)
    else:
        _,l,tr_pred = sess.run([sgd_optimize,loss,train_prediction], feed_dict=feed_dict)
        
    return l, tr_pred

### Defining Data Generators and Other Related Variables

In [60]:
# This is where all the results will be logged into
log_dir = 'logs'
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

# Filenames of the logs
train_prediction_text_fname = 'train_predictions.txt'
test_prediction_text_fname = 'test_predictions.txt'

# Some configuration for the TensorFlow session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement=True
sess = tf.InteractiveSession(config=config)

# Initialize variables
tf.global_variables_initializer().run()

# Load the word embeddings
src_word_embeddings = np.load('de-embeddings.npy')
tgt_word_embeddings = np.load('en-embeddings.npy')

# Defining data generators
def define_data_generators(batch_size, enc_num_unrollings, dec_num_unrollings):
    
    # Training data generators (Encoder and Decoder)
    enc_data_generator = DataGeneratorMT(batch_size=batch_size,num_unroll=enc_num_unrollings,is_source=True, is_train=True)
    dec_data_generator = DataGeneratorMT(batch_size=batch_size,num_unroll=dec_num_unrollings,is_source=False, is_train=True)

    # Testing data generators (Encoder and Decoder)
    test_enc_data_generator = DataGeneratorMT(batch_size=batch_size,num_unroll=enc_num_unrollings,is_source=True, is_train=False)
    test_dec_data_generator = DataGeneratorMT(batch_size=batch_size,num_unroll=dec_num_unrollings,is_source=False, is_train=False)
    
    return enc_data_generator,dec_data_generator,test_enc_data_generator,test_dec_data_generator

### Run

In [61]:
# Training and test BLEU scores
train_bleu_scores_over_time,test_bleu_scores_over_time = [],[]
# Loss over time
loss_over_time = []

# Labels and predictions required to calculate the BLEU scores
# For both train and test data
train_bleu_refs, train_bleu_cands = [],[]
test_bleu_refs, test_bleu_cands = [],[]

# Number of steps the training goes for
num_steps = 10001
avg_loss = 0

# Defining data generators for encoder/decoder and training/testing
enc_data_generator, dec_data_generator, \
test_enc_data_generator, test_dec_data_generator = \
define_data_generators(batch_size, enc_num_unrollings, dec_num_unrollings)

print('Started Training')

for step in range(num_steps):

    # input (encoder) unrolling length: 40
    # output (decoder) unrolling length: 60
    if (step+1)%100==0:
        print('.',end='')

    # Sample a random batch of IDs from training data
    sent_ids = np.random.randint(low=0,high=train_inputs.shape[0],size=(batch_size))
    
    # Getting an unrolled set of data batches for the encoder
    eu_data, _, _ = enc_data_generator.unroll_batches(sent_ids=sent_ids)
    
    # Getting an unrolled set of data batches for the decoder
    du_data, du_labels, _ = dec_data_generator.unroll_batches(sent_ids=sent_ids)
    
    # Train for single step
    l, tr_pred = train_single_step(eu_data, du_data, du_labels)
    
    # We don't calculate BLEU scores all the time as this is expensive, 
    # it slows down the code
    if np.random.random()<0.1:
        
        # all_labels are labels obtained by concatinating all the labels in batches
        all_labels = np.argmax(np.concatenate(du_labels,axis=0),axis=1)
        # all_preds are predictions for all unrolled steps
        all_preds = np.argmax(tr_pred,axis=1)
        
        # Get training BLEU candidates and references
        batch_cands, batch_refs = create_bleu_ref_candidate_lists(all_preds, all_labels)
        
        # Accumulate training candidates/references for calculating
        # BLEU later
        train_bleu_refs.extend(batch_refs)
        train_bleu_cands.extend(batch_cands)

    if (step+1)%500==0:  
        
        # Writing actual and predicte data to train_prediction.txt file for some random sentence
        print('Step ',step+1)
        with open(os.path.join(log_dir, train_prediction_text_fname),'a') as fa:                                
            fa.write('============= Step ' +  str(step+1) + ' =============\n') 
        
        rand_idx = np.random.randint(low=1,high=batch_size)
        print_and_save_train_predictions(du_labels, tr_pred, rand_idx, train_prediction_text_fname)        
        
        # Calculating the BLEU score for the accumulated candidates/references
        bscore = 0.0
        bscore = corpus_bleu(train_bleu_refs,train_bleu_cands,smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method4)
        train_bleu_scores_over_time.append(bscore)
        print('(Train) BLEU (%d elements): '%(len(train_bleu_refs)),bscore)
        
        # Reset the candidate/reference accumulators
        train_bleu_refs, train_bleu_cands = [],[]
        
        # Write BLEU score to file
        with open(log_dir + os.sep +'blue_scores.txt','a') as fa_bleu:
            fa_bleu.write(str(step+1) +','+str(bscore)+'\n')
        
        with open(os.path.join(log_dir, train_prediction_text_fname),'a') as fa:                
            fa.write('(Train) BLEU: %.5f\n'%bscore)        
        
    avg_loss += l  # Update average loss
    
    sess.run(reset_train_state) # Resetting hidden state for each batch
    
    # ============================= TEST PHASE ==================================
    if (step+1)%1000==0:
        # Calculate average loss
        print('============= Step ', str(step+1), ' =============')
        print('\t Loss: ',avg_loss/1000.0)
        loss_over_time.append(avg_loss/1000.0)
        
        # Write losses to file
        with open(log_dir + os.sep + 'losses.txt','a') as fa_loss:
            fa_loss.write(str(step+1) +','+str(avg_loss/1000.0)+'\n')
        
        with open(os.path.join(log_dir, train_prediction_text_fname),'a') as fa:                                
            fa.write('============= Step ' +  str(step+1) + ' =============\n') 
            fa.write('\t Loss: %.5f\n'%(avg_loss/1000.0))
            
        avg_loss = 0.0
        
        # Increase gstep to decay learning rate
        sess.run(inc_gstep)
        
        # reset global step when we change the optimizer
        if (step+1)==20000: 
            sess.run(reset_gstep)
        
        print('=====================================================')
        print('(Test) Translating test sentences ...')        

        print('Processing test data ... ')
        
        # ===================================================================================
        # Predictions for Test data
        for in_i in range(test_inputs.shape[0]//batch_size):
            
            # Generate encoder / decoder data for testing data
            test_eu_data, test_eu_labels, _ = test_enc_data_generator.unroll_batches(sent_ids=np.arange(in_i*batch_size,(in_i+1)*batch_size))
            test_du_data, test_du_labels, _ = test_dec_data_generator.unroll_batches(sent_ids=np.arange(in_i*batch_size,(in_i+1)*batch_size))
            
            # fill the feed dict
            feed_dict = {}
            for ui,(dat,lbl) in enumerate(zip(test_eu_data,test_eu_labels)):            
                feed_dict[enc_test_input[ui]] = dat             

            # Get predictions out with decoder          
            # run prediction calculation this returns a list of prediction dec_num_unrollings long
            test_pred_unrolled = sess.run(test_predictions, feed_dict=feed_dict)
            
            # We print a randomly selected sample from each batch
            test_rand_idx = np.random.randint(0,batch_size) # used for printing test output
            
            print_and_save_test_predictions(test_du_labels, test_pred_unrolled, in_i, test_rand_idx, test_prediction_text_fname)
            
            # Things required to calculate test BLEU score
            all_labels = np.argmax(np.concatenate(test_du_labels,axis=0),axis=1)
            all_preds = np.concatenate(test_pred_unrolled, axis=0)
            batch_cands, batch_refs = create_bleu_ref_candidate_lists(all_preds, all_labels)
            test_bleu_refs.extend(batch_refs)
            test_bleu_cands.extend(batch_cands)
            
            # Reset the test state
            sess.run(reset_test_state)
        
        # Calculate test BLEU score
        test_bleu_score = 0.0
        test_bleu_score = corpus_bleu(test_bleu_refs,test_bleu_cands,
                                      smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method4)
        test_bleu_scores_over_time.append(test_bleu_score)
        print('(Test) BLEU (%d elements): '%(len(test_bleu_refs)),test_bleu_score)
        
        test_bleu_refs, test_bleu_cands = [],[]        
        print('=====================================================')

Started Training
.....Step  500
Actual: Joining France &apos;s &quot; Big Three &quot; in the NBA are four more young talents : <unk> <unk> of the L <unk> . Lakers ; <unk> <unk> of the Denver <unk> ; and Johan <unk> and <unk> <unk> of the Seattle <unk> . </s> 

Predicted: The <unk> , <unk> <unk> , <unk> , the <unk> , the , , , , the , , the <unk> , , </s> 
(Train) BLEU (510 elements):  0.06707996439724499
.....Step  1000
Actual: The guest reviews are submitted by our customers after their stay at The Oriental Siam Resort . </s> 

Predicted: When guest reviews are submitted by our customers after their stay at <unk> <unk> Hotel . . </s> 
(Train) BLEU (490 elements):  0.17890726789378786
============= Step  1000  =============
	 Loss:  1.211373913615942
(Test) Translating test sentences ...
Processing test data ... 
DE:  Der nordwestliche Teil der Insel besteht aus Granit und Gneis , von Ton überlagert , und bildet eine ca.

	 EN (TRUE):A battle between Denmark and Sweden in 1645 led to 